In [31]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


llm = ChatOpenAI(
    # model="gemma",
    temperature=0.1,
)

memory = ConversationBufferMemory(
    llm=llm,
    # max_token_limit=120,
    return_messages=True,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/1984.txt")

docs = loader.load_and_split(text_splitter=splitter)

# # loader.load()
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know. don't make it up.:\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
def load_memory(_):
    return memory.load_memory_variables({})["history"]

retriver = vectorstore.as_retriever()

chain = {"context": retriver, "question": RunnablePassthrough(), "history":load_memory} | prompt | llm

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input":question},
        {"output":result.content},
    )
    print(result)


In [32]:
invoke_chain("Is Aaronson guilty?")

content='Yes, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [33]:
invoke_chain("What message did he write in the table?")

content='He wrote "2+2=5" on the table.'


In [34]:
invoke_chain("Who is Julia?")

content='Julia is a character in the story who is involved in a romantic relationship with the protagonist.'
